<a href="https://colab.research.google.com/github/istiakm30/File-and-Torrent-Downloader/blob/main/Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Files

In [ ]:
#@title Basic Downloader

print('Importing necessary libraries...')
try:
  from tqdm.notebook import tqdm
except:
  !pip install tqdm > /dev/null

import requests
from tqdm.notebook import tqdm
from google.colab import files
import re
print('DONE')
# Create a GUI to input the URL
download_link = '' #@param {type:"string"}

# Get the file size
response = requests.get(download_link, stream=True)
total_size = int(response.headers.get('content-length', 0))

# Extract the filename from the Content-Disposition header
d = response.headers['content-disposition']
filename = re.findall("filename=(.+)", d)[0]

# Initialize a progress bar
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Download the file with progress updates
with open(filename, 'wb') as file:
    for data in response.iter_content(chunk_size=1024):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()

if total_size != 0 and progress_bar.n != total_size:
    print("ERROR: something went wrong with the download.")
else:
    print(f"{filename} downloaded successfully.")

# Create a download link in Colab
files.download(filename)


In [ ]:
#@title Multi-Threaded Downloader

print('Importing necessary libraries...')

# Import necessary libraries
!pip install pySmartDL > /dev/null
from pySmartDL import SmartDL
from google.colab import files
from tqdm.notebook import tqdm
import os

print('DONE')

# Create a GUI to input the URL
download_link = '' #@param {type:"string"}

# Get the current working directory
cwd = os.getcwd()

# Create a SmartDL object
obj = SmartDL(download_link, dest=cwd, progress_bar=False)

# Start the download in a separate thread
obj.start(blocking=False)

# Create a progress bar
progress_bar = tqdm(total=obj.filesize, unit='B', unit_scale=True, unit_divisor=1024)

# Update the progress bar in a loop
while not obj.isFinished():
    progress_bar.update(obj.get_dl_size() - progress_bar.n)

# Close the progress bar
progress_bar.close()

# Wait for the download to finish
obj.wait()

# Get the filename
filename = os.path.basename(obj.get_dest())

# Check if the download was successful
if obj.isSuccessful():
    print(f"{filename} downloaded successfully.")
else:
    print("ERROR: something went wrong with the download.")

# Create a download link in Colab
files.download(os.path.join(cwd, filename))

In [ ]:
#@title Move the Downloaded File into `Google Drive's "Downloaded"` Folder

from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Path to the folder where you want to save the file in your Google Drive
destination_folder = "/content/gdrive/MyDrive/Downloaded/"

# Ensure the folder exists
os.makedirs(destination_folder, exist_ok=True)

# Move the downloaded file to the Google Drive folder
shutil.move(filename, destination_folder + filename)

print(f"{filename} copied successfully to {destination_folder}")
print("File has been copied to the 'Downloaded' folder in your Google Drive.")
print("Please navigate to this folder in your Google Drive and create a shareable link if you wish to share access.")

# Download Torrents

In [ ]:
#@title Torrent File Downloader with `.torrent` File
print('Importing necessary libraries...')

# Install libtorrent and tqdm
try:
  import libtorrent as lt
except:
  !apt install -y python3-libtorrent > /dev/null
try:
  from tqdm.notebook import tqdm
except:
  !pip install tqdm > /dev/null
print('DONE')

import os
import libtorrent as lt
import time
from google.colab import files
from tqdm.notebook import tqdm

# Upload the torrent file
uploaded_torrent_file = files.upload()
torrent_file_path = list(uploaded_torrent_file.keys())[0]

# Create a hidden folder
hidden_folder_path = './.hidden_folder'
os.makedirs(hidden_folder_path, exist_ok=True)

# Move the uploaded file into the hidden folder
new_torrent_file_path = os.path.join(hidden_folder_path, torrent_file_path)
os.rename(torrent_file_path, new_torrent_file_path)

# Now, the torrent file is in the hidden folder
torrent_file_path = new_torrent_file_path

# Create a session
ses = lt.session()
info = lt.torrent_info(torrent_file_path)
h = ses.add_torrent({'ti': info, 'save_path': '.'})

# Get torrent info
total_size = info.total_size()

# Initialize progress bar
progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

# Downloading progress
print('Downloading: '+ '\033[1;32m' + '' + h.torrent_file().name() + '\033[0m')

s = h.status()
while not s.is_seeding:
    downloaded_size = s.total_done
    progress_bar.update(downloaded_size - progress_bar.n)
    peers = s.num_peers
    print('\033[1;36m' + f'\rNumber of peers: {peers}' + '\033[0m', end='')
    time.sleep(1)
    s = h.status()

print()
progress_bar.close()
print('\033[1;32m' + '\nDownload Complete. You Can Find Your File in the Folder Named "Downloaded" at "https://drive.google.com/drive/"' + '\033[0m')

In [ ]:
#@title Torrent File Downloader with `Magnet Link`

# Install libtorrent and tqdm
print('Importing necessary libraries...')

try:
  import libtorrent as lt
except:
  !apt install -y python3-libtorrent > /dev/null
try:
  from tqdm.notebook import tqdm
except:
  !pip install tqdm > /dev/null

import libtorrent as lt
import time
from tqdm.notebook import tqdm
print('DONE')


# Input the magnet link
magnet_link = "" #@param {type:"string"}

# Create a session
ses = lt.session()

# Add magnet link as a torrent
info_hash = lt.parse_magnet_uri(magnet_link)
info_hash.save_path = './'  # Set the path where to save the download
h = ses.add_torrent(info_hash)

# Wait for metadata to download
print('Downloading Metadata...')
while not h.torrent_file():
    time.sleep(1)
print('Metadata Downloaded')

# Get torrent info
info = h.torrent_file()
total_size = info.total_size()

# Initialize progress bar
progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

# Downloading progress
print('Downloading: '+ '\033[1;36m' + '' + info.name() + '\033[0m')

s = h.status()
while not s.is_seeding:
    downloaded_size = s.total_done
    progress_bar.update(downloaded_size - progress_bar.n)
    peers = s.num_peers
    print(f'\rNumber of peers: {peers}', end='')  # Use carriage return to overwrite the current line
    time.sleep(1)
    s = h.status()

print()  # Print a newline character after the loop to ensure proper formatting
progress_bar.close()
print('\033[1;32m' + '\nDownload Complete. ' + '\033[0m')

In [ ]:
#@title Copy the Torrent File into `Google Drive's "Downloaded"` Folder

from google.colab import drive
import shutil
import os

drive.flush_and_unmount()
drive.mount('/content/gdrive', force_remount=True)

source = './' + h.torrent_file().name()
destination = '/content/gdrive/MyDrive/Downloaded/' + h.torrent_file().name()

if os.path.isfile(source):
    try:
        new_location = shutil.copy2(source, destination)
        print(f"The file has been copied to: {os.path.join(destination, os.path.basename(new_location))}")
        print("You Can Find Your File in the Folder Named \"Downloaded\" at \"https://drive.google.com/drive/\"")
    except FileExistsError:
        print(f"File already exists at the destination. Please remove the file from {destination} and try again.")
elif os.path.isdir(source):
    try:
        shutil.copytree(source, destination)
        print(f"The directory has been copied to: {destination}")
    except FileExistsError as e:
        print(f"Directory already exists at the destination. Please remove the File/Directory called \"{h.torrent_file().name()}\" from {destination} and try again.")
else:
    print("The source is not a valid file or directory.")

# File Removing Section. You Can Remove the Files from Google Drive.
---
(Do Not Run 2 Cells)


In [ ]:
#@title Remove the Downloaded File from Drive (Just in Case)
import os

try:
    # specify the path to the file
    file_path = f"/content/drive/MyDrive/Downloaded/{filename}"

    # check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
        print("File removed successfully")
    else:
        print("The file does not exist")

except NameError:
    print("The variable 'filename' is not defined. Did you run the File Downloader?")

In [ ]:
#@title Remove the Torrent Files from Drive (Just in Case)
import os
try:
  # specify the path to the file
  file_path = f"/content/drive/MyDrive/Downloaded/{h.torrent_file().name()}"

  # check if the file exists
  if os.path.exists(file_path):
      # remove the file
      os.remove(file_path)
      print("File removed successfully")
  else:
      print("The file does not exist")
except:
  print("Did you Download a Torrent File First?")